### Train a Noise2Void model

Both the CARE network and Noise2Noise network you trained in part 1 and 2 require that you acquire additional data for the purpose of denoising. For CARE we used a paired acquisition with high SNR, for Noise2Noise we had paired noisy acquisitions. We will now train a Noise2Void network from single noisy images.

This notebook uses a single image from the SEM data from the Noise2Noise notebook.

We use the [Careamics](https://careamics.github.io) library

<div class="alert alert-block alert-danger\"> 
Set your kernel to 
</div>




<div class="alert alert-block alert-info"><h3>Task 2.2: Bla</h3>
    
This notebook uses a single image from the SEM data from the Noise2Noise notebook.

If you brought your own raw data, use that instead! The only requirement is that the noise in your data is pixel-independent and zero-mean. If you're unsure whether your data fulfills that requirement or you don't yet understand why it is necessary ask one of us to discuss!

If you don't have suitable data of your own, feel free to find some online or ask your fellow course participants. You can however also stick with the SEM data provided here and compare the results to what you achieved with Noise2Noise in the previous part.


</div>

![diff_costs](static/diff_costs.png)

<hr style="height:2px;"><div class="alert alert-block alert-success"><h1>Checkpoint 1</h1>

</div>

In [ ]:
import shutil
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import tifffile
from careamics_portfolio import PortfolioManager

from careamics import CAREamist
from careamics.config import (
    create_n2v_configuration,
)
from careamics.transforms import N2VManipulate

### Part 1. Prepare the data

For this we download the relevant dataset from the CAREamics portfolio library

In [ ]:
# Explore portfolio
portfolio = PortfolioManager()
print(portfolio.denoising)

In [ ]:
# Download files # TODO File should be reused from previous exercise
root_path = Path("./data")
files = portfolio.denoising.N2V_SEM.download(root_path)
print(f"List of downloaded files: {files}")

#### Visualize training data

In [ ]:
# Load images
train_image = tifffile.imread(files[0])
print(f"Train image shape: {train_image.shape}")
plt.imshow(train_image, cmap="gray")

#### Visualize validation data

In [ ]:
val_image = tifffile.imread(files[1])
print(f"Validation image shape: {val_image.shape}")
plt.imshow(val_image, cmap="gray")

In [ ]:
# Set paths

data_path = Path(root_path / "n2v_sem")
train_path = data_path / "train"
val_path = data_path / "val"

train_path.mkdir(parents=True, exist_ok=True)
val_path.mkdir(parents=True, exist_ok=True)

shutil.copy(root_path / files[0], train_path / "train_image.tif")
shutil.copy(root_path / files[1], val_path / "val_image.tif")

<div class="alert alert-block alert-info"><h3>Part 2: Learn about N2V masking</h3>

To keep the network from learning the identity we have to manipulate the input pixels for the blindspot during training. How to exactly manipulate those values is controlled via the N2VManipulate and strategy to be specific. Default value of the masking strategy parameter is 'uniform' which samples a random value from the surrounding pixels, including or excluding the value at the control point. The size of the surrounding area can be configured via roi_size, which defines the side of a square region around the control point. 

The paper supplement describes other pixel manipulators as well (section 3.1). 
</div>

#### Part 2.1 Visualize masking algorithm

Let's start by looking into how Noise2Void masking works. The default masking strategy
replaces certain percentage of pixel values with randomly selected values from the vicinity.

In the next cell we'll define the parametes. Feel free to play around with the values to see how the masking changes.

In [ ]:
# Define masking parameters
dummy_patch_size = 10
roi_size = 5
masked_pixel_percentage = 5
strategy = 'uniform'

# This is simply the index of a pixel to draw a rectangle around
highlighted_pixel_idx = 0

In [ ]:
# Create a dummy patch
patch = np.arange(dummy_patch_size**2).reshape(dummy_patch_size, dummy_patch_size)

In [ ]:
# instantiate the manipulator
manipulator = N2VManipulate(
    roi_size=roi_size,
    masked_pixel_percentage=masked_pixel_percentage,
    strategy=strategy,
)

# We need to add channel dimension because it's expected by the manipulator
masked_patch, _, mask = manipulator.apply(patch[..., None])


#### Define helper variables for visualization

In [ ]:
# Get the indices of the masked pixels
i, j = np.where(mask.squeeze())
masked_pixel_coords = np.concatenate([i[:, None], j[:, None]], axis=1)

# Get the coordinates of the one of the border pixels
center_coords = np.flip(masked_pixel_coords[highlighted_pixel_idx]) - roi_size // 2

#### Visualize the pixel with replaced values and the mask

The cell below will visualize the patch with replaced values and the corresponding mask.
You can see the size of the region of interest (roi_size) defined by the red square. The pixel in the center of the square is the one that was replaced with a random value from within the square.

In [ ]:
# Define a roi mask to visualize the masked region
roi = plt.Rectangle(center_coords, roi_size - 1, roi_size - 1, edgecolor='r', fill=False)

# Visualize the masked patch and the mask
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(masked_patch)
ax[0].add_patch(roi)
ax[1].imshow(mask, cmap="gray")

<div class="alert alert-block alert-info"><h3>Part 2.2: Create configuraion</h3>

CAREamics can be configured either from a yaml file, or with an explicitly created config object.
In this note book we will create the config object using the helper function. CAREamics will 
validate all the parameters and will output explicit error if some parameters or a combination of parameters isn't allowed. It will also provide default values for missing parameters. 

When creating the config-object, we provide the training data X. From X the library will extract mean and std that will be used to normalize all data before it is processed by the network.

For Noise2Void training it is possible to pass arbitrarily large patches to the training method. From these patches random subpatches of size patch_size are extracted during training. Default patch shape is set to (64, 64).
Parameters axes and data_type control the shape of data and the extension of the file.
It's also possible to provide 'array' as data_type, in which case the data is expected to be a numpy array.

Please take as look at the [documentation](https://careamics.github.io) to see the full list of parameters and configuration options

</div>

<div class="alert alert-block alert-info"><h3>Task 2.2: Bla</h3>

Take a look at the configuration parameters, make sure you understand what they do and 
try to run the training.

</div>


In [ ]:
# Create a configuration using the helper function
training_config = create_n2v_configuration(
    experiment_name="LevitatingFrog",
    data_type="tiff",
    axes="YX",
    patch_size=[64, 64],
    batch_size=128,
    num_epochs=10,
    roi_size=3,
    masked_pixel_percentage=0.05,
)

#### Initialize the Model

Let's instantiate the model with the configuration we just created.

CAREamist is the main umbrella class of the library, which will handle creation of the data pipeline, the model, training and inference methods.

It's also possible to define all the components separately, which will allow for more fine-grained control over the data preparation and training processes. CAREamics is using Pytorch Lightning, so defining the pipeline will follow the regular Pytorch Lightning procedure.

Please take as look at the [documentation](https://careamics.github.io) to see the full description

In [ ]:
engine = CAREamist(source=training_config)

### Part 3. Run training 

We need to specify the paths to training and validation data. 

CAREamics can also accept an array or a list of arrays, in which case the data is expected to fit in memory. In case of paths or Pytorch Lightning datamodule data can be processed in memory or streamed from disk. 

Please take as look at the [documentation](https://careamics.github.io) to see the full description of supported data types

In [ ]:
engine.train(train_source=train_path, val_source=val_path)

### Part 4. Prediction

For performing inference, we also need to specify the path and we can use the bigger tile size to make the process faster. 

By default CAREamics uses tiled prediction to handle large images. The tile size can be set via the tile_size parameter. Tile overlap is computed automatically based on the network architecture.

To see the full description of the parameters and methods, please take a look at the [documentation](https://careamics.github.io)

In [ ]:
preds = engine.predict(source=train_path, tile_size=(256, 256))

### Visualize predictions

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(train_image, cmap="gray")
ax[1].imshow(preds.squeeze(), cmap="gray")

In [ ]:
# Zoom in

y_start = 100
y_end = 400
x_start = 100
x_end = 400

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(train_image[y_start:y_end, x_start:x_end], cmap="gray")
ax[1].imshow(preds.squeeze()[y_start:y_end, x_start:x_end], cmap="gray")

<div class="alert alert-block alert-info"><h3>Task 3: Try to improve results</h3>

CAREamics configuration won't allow you to use parameters which are clearly wrong. However, there are many parameters that can be tuned to improve the results. Try to play around with the roi_size and masked_pixel_percentage and see if you can improve the results.

</div>
